# Layers and Blocks

We now use code to generates a network with one fully-connected hidden layer with 256 units and ReLU activation, followed by a fully-connected output layer with 10 units.(MLP)

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [4]:
net = nn.Sequential(
            nn.Linear(20, 256),
            nn.ReLU(),
            nn.Linear(256,10)
                    )
# initial a X-net
X = torch.rand(2,20)
net(X)

tensor([[-0.1090,  0.2617,  0.1134, -0.0333,  0.1857, -0.1067,  0.0774, -0.1264,
         -0.0744,  0.1567],
        [-0.0768,  0.1188,  0.1009,  0.0162,  0.0447, -0.0759,  0.0224, -0.1228,
         -0.0717,  0.1000]], grad_fn=<AddmmBackward0>)

### A Custom Block

In the following snippet, we code up a block from scratch corresponding to an MLP with one hidden layer with 256 hidden units, and a 10-dimensional output layer.
Note that the MLP class below inherits the class that represents a block. We will heavily rely on the parent class's functions, supplying only our own constructor and forward propagation function.

In [5]:
class MLP(nn.Module):
    def __init__(self):
        # Call the constructor of the MLP parent class 'Module' to perform the necessary initialization.
        # In this way, other function arguments can also be specified during class instantiation, such as the model parameters
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [6]:
net = MLP()
net(X)

tensor([[ 0.0905,  0.0689, -0.0605,  0.0758, -0.0441, -0.0462,  0.0725, -0.3150,
          0.2012,  0.0401],
        [-0.0036, -0.0122, -0.0459,  0.0017,  0.0017, -0.0534,  0.0480, -0.2251,
          0.1995, -0.0778]], grad_fn=<AddmmBackward0>)

### The Sequential Block

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # When using the iterators, we need to keep track of the number of items in the iterator. 
            # This is achieved by an in-built method called enumerate(). 
            # The enumerate() method adds counter to the iterable.
            # 'module' is an instance of a 'Module' subclass
            self._modules[str(idx)] = module
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [8]:
net = MySequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))
net(X)

tensor([[-0.0375,  0.1438, -0.2824,  0.0218, -0.2226,  0.0686, -0.0477,  0.1067,
         -0.1525, -0.0817],
        [ 0.0385,  0.1697, -0.2082,  0.0510, -0.1613,  0.0483,  0.0279,  0.0639,
         -0.2172, -0.0457]], grad_fn=<AddmmBackward0>)

### Executing Code in the Forward Propagation Function

The Sequential class makes model construction easy, allowing us to assemble new architectures without having to define our own class. The following implement a FixedHiddenMLP class as follows.

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and therefore keep constant during training
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1) #torch.mm Matrix mutiple
        # Reuse the fully-connected layer. This is equivalent to sharing parameters with two fully-connected layers
        X = self.linear(X)
        # Control Flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [10]:
net = FixedHiddenMLP()
net(X)

tensor(0.0194, grad_fn=<SumBackward0>)

 We can mix and match various ways of assembling blocks together. In the following example, we nest blocks in some creative ways.

In [14]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64), nn.ReLU(),
                                 nn.Linear(64,32), nn.ReLU())
        
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0877, grad_fn=<SumBackward0>)

### Conclusion

~Layers are blocks.
~Many layers can comprise a block.
~Many blocks can comprise a block.
~A block can contain code.
~Blocks take care of lots of housekeeping, including parameter initialization and backpropagation.
~Sequential concatenations of layers and blocks are handled by the Sequential block.